In [2]:
import json
import numpy as np
import pandas as pd

In [3]:
import sys
sys.path.append(r'C:/Users/vshnu/OneDrive/Desktop/Vishnuperson/RE/aspire_')
from connector import data_fetch_v2

In [ ]:
billing_data = f"""
select billing_cycle, user_id, dpd, principal_outstanding
from billing_info """
flag,billing_data_df, error_msg = data_fetch_v2(billing_data, host="13.126.97.63")

In [ ]:
billing_data_df.dtypes

In [ ]:
billing_data1 = f"""
select billing_cycle, user_id, dpd, principal_outstanding
from billing_info """
flag,billing_data_df2, error_msg = data_fetch_v2(billing_data1, host="13.126.97.63")

In [42]:
def increment_period_series(series: pd.Series) -> pd.Series:
    return series.astype(str).apply(lambda x: f"{int(x[:4]) + 1}01" if x.endswith("12") else x.replace(x[4:], str(int(x[4:]) + 1).zfill(2)))


billing_data_df2['billing_cycle_next'] = increment_period_series(billing_data_df2['billing_cycle'])

In [38]:
#x = "202412"
x1 = "202404"
#y  = f"{int(x[:4])+1}01"
#y = f"{x1[:4]}{int(x1[4:]):02d}"
y = x1.replace(x1[4:], str(int(x1[4:])+1).zfill(2))
print(y)

202405


In [49]:
billing_data_df2 = billing_data_df2.drop(columns=['principal_outstanding'])

In [45]:
billing_data_df2 = billing_data_df2.rename(columns={'dpd':'next_dpd'})

In [ ]:
billing_data_df2

In [46]:
billing_data_df["dpd_bucket"] = billing_data_df["dpd"].apply(lambda x: "A: current" if x == 0 else
                                                             "B: DPD_1_30" if x>=1 and x<=30 else
                                                             "C: DPD_31_60" if x>=31 and x<=60 else
                                                             "D: DPD_61_90" if x>=61 and x<=90 else
                                                             "E: DPD_91_180" if x>=91 and x<=180 else ">180")

In [ ]:
billing_data_df

In [54]:
ce_calc = billing_data_df.merge(billing_data_df2, left_on=("user_id","billing_cycle"), right_on=("user_id","billing_cycle_next"),how="left")

In [ ]:
ce_calc.dtypes

In [61]:
ce_calc["next_dpd"] = ce_calc["next_dpd"].fillna(0).astype(int)

In [77]:
def ce_def(a,b):
    if a-b>0:
        return "roll_back/dpd_decrease"
    elif a-b<0:
        return "roll_forwards/dpd_increase"
    else:
        return "stab"

In [78]:
ce_calc["collection_eff"] = ce_calc.apply(lambda row: ce_def(row["dpd"], row["next_dpd"]), axis=1)

In [ ]:
ce_calc

In [ ]:
ce_calc.dtypes

In [ ]:
ce_calc.agg('collection_eff').value_counts()

In [113]:
pivot.head(5)

collection_eff,roll_back/dpd_decrease,roll_forwards/dpd_increase,stab
billing_cycle,,,
202104,5,0,233
202105,52,0,236
202106,79,6,243
202107,110,4,255
202108,115,7,311


In [114]:
pivot["total"] = pivot["roll_back/dpd_decrease"]+pivot["roll_forwards/dpd_increase"]+pivot["stab"]

In [115]:
pivot["roll_back%"] = round((pivot["roll_back/dpd_decrease"]/pivot["total"])*100,2)
pivot["roll_forward%"] = round((pivot["roll_forwards/dpd_increase"]/pivot["total"])*100,2)
pivot["stab%"] = round((pivot["stab"]/pivot["total"])*100,2)

In [110]:
pivot = ce_calc.groupby(['billing_cycle',"collection_eff"])["user_id"].count().unstack(fill_value=0)


In [ ]:
pivot